# This notebook shall serve as a base to experiment with simple API calls from the DocuCam API.

Objectives, write scripts for:

* Obtain an image for a specific timestamp. 
* Obtain images for a specific time range (1day+)
* Understand the data structure --> Full Images and their respective layers (500?). How to obtain them.
* Obtain image + layers or only specific layers. 

In the end, we would want to have a script that takes some parameters as input (time range, location, quality, etc) and outputs a dataframe that combines both: Measurements per timestamp and the image plus the respective layers per timestamp. Images may be appended as simple numerical representations OR in the first step simply provide path to were image is stored. 

Problem: time stamps not in sync. Need to create a costum common identifier? 
